<a href="https://colab.research.google.com/github/buzzbing/buzzbing.github.io/blob/master/ServiceRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:


import numpy as np
import pandas as pd
import sklearn
import collections
from sklearn.model_selection import train_test_split
# from mpl_toolkits.mplot3d import Axes3D
# from IPython import display
# from matplotlib import pyplot as plt

# import sklearn.manifold
import tensorflow as tf

In [3]:
dataset = pd.read_excel('/content/DatasetForMP.xlsx')

In [4]:
dataset.head()
dataset = dataset.sample(frac=1)
dataset.head()

,uid,lat,lng,serviceIDs,orderATime,orderId,itemID,title
58,87o9MiaNURUUR0l17FzIdJ126a92,27.712652,85.294236,1678696474585,21-02-2023 05:46:39 PM,1678708899085,1678696474585,Pest Cleaning
37,ZXQnDNWjaxQjqMuIlZiMWlewgMi2,27.688812,85.289732,1677414713802,13-02-2023 02:05:14 PM,1678695614506,1677414713802,Hardware
50,janhPjRaLEVGupblcIRlZcFWEMK2,27.678558,85.304616,1678696474585,15-02-2023 03:21:08 PM,1678700168482,1678696474585,Pest Cleaning
56,TjVQX9vBhXW1cHkBsntSC2FImZi1,27.687642,85.288394,1678696318063,21-02-2023 05:45:08 PM,1678708808387,1678696318063,Room Cleaning
85,OOPrmYZDwyYlelfVSTsdUjzN6oz2,27.712656,85.294237,1678696416054,28-02-2023 06:19:55 PM,1678710895559,1678696416054,Gardening Service


In [5]:
dataset['uid'].nunique()

40

In [33]:
y = dataset['itemID'].astype(str)

In [34]:
y.head()

58    1678696474585
37    1677414713802
50    1678696474585
56    1678696318063
85    1678696416054
Name: itemID, dtype: object

In [35]:
y = pd.DataFrame(y)

In [36]:
print(type(y))

<class 'pandas.core.frame.DataFrame'>


In [37]:
X = dataset.drop(columns=['title','serviceIDs','itemID','uid','orderId','uid','orderATime'])
# X = dataset.drop(columns=['title'])

In [38]:
X.head()

,lat,lng
58,27.712652,85.294236
37,27.688812,85.289732
50,27.678558,85.304616
56,27.687642,85.288394
85,27.712656,85.294237


In [39]:
# X['orderATime']= all_data['Order Day new'].apply(lambda x: dt.datetime.strftime(x, '%Y-%m-%d'))

# X['orderATime'] = X['orderATime'].astype(str)

In [40]:
X.head()

,lat,lng
58,27.712652,85.294236
37,27.688812,85.289732
50,27.678558,85.304616
56,27.687642,85.288394
85,27.712656,85.294237


In [41]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = 0.9,test_size = 0.1,random_state =10)

In [42]:
print(X_train.columns,end='\n\n')
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
print(object_cols)

Index(['lat', 'lng'], dtype='object')

[]


In [43]:
object_nunique = list(map(lambda col: X_train[col].nunique(),object_cols))
d = dict(zip(object_cols,object_nunique))
sorted(d.items(),key=lambda x:x[1])

[]

In [44]:
train_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]
missing_label_cols = list(set(object_cols)-set(train_label_cols))
print(missing_label_cols)


[]


In [45]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

label_X_train[object_cols] = encoder.fit_transform(label_X_train[object_cols])
label_X_valid[object_cols] = encoder.transform(label_X_valid[object_cols])

label_X_train.head()


,lat,lng
50,27.678558,85.304616
37,27.688812,85.289732
54,27.687642,85.288394
41,27.688815,85.289723
15,27.712632,85.294196


In [46]:
y.head()

,itemID
58,1678696474585
37,1677414713802
50,1678696474585
56,1678696318063
85,1678696416054


In [59]:

_cols = [col for col in y_train.columns if y_train[col].dtype == 'object']
print(_cols)

['itemID']


In [60]:
encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

label_y_train = y_train.copy()
label_y_valid = y_valid.copy()

label_y_train[_cols] = encoder.fit_transform(label_y_train[_cols])
label_y_valid[_cols] = encoder.transform(label_y_valid[_cols])


print(label_y_train)


    itemID
50    11.0
37     6.0
54     9.0
41     0.0
15    14.0
..     ...
59    10.0
10     3.0
73    10.0
5     13.0
75     4.0

[89 rows x 1 columns]


In [61]:
label_y_train.head()

,itemID
50,11.0
37,6.0
54,9.0
41,0.0
15,14.0


In [62]:
# from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# model = RandomForestClassifier(n_estimators = 200, random_state = 10)

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=5)

model.fit(label_X_train,label_y_train.itemID)
preds = model.predict(label_X_valid)




In [63]:
print(mean_absolute_error(label_y_valid.itemID,preds))

4.2


In [64]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(label_y_valid,preds )
print(accuracy)


0.0


In [65]:
label_X_valid.head()

,lat,lng
40,27.688804,85.289698
4,27.712638,85.294201
30,27.690684,85.310214
21,27.712625,85.294190
9,27.712638,85.294201


In [66]:
# ordertime = encoder.inverse_transform(label_X_train['orderATime'])

In [67]:
t=np.array([[27.700652      ,85.191222      ,12.0]])

In [68]:
t.reshape((-1,1))

array([[27.700652],
       [85.191222],
       [12.      ]])

In [79]:
x =  int(model.predict([[27.6783400	,85.2883936]]))
  

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [80]:
print(encoder.inverse_transform(np.array(x).reshape(-1,1)))

[['1677414713802']]


In [91]:
import pickle

pickle.dump(model,open('drive/MyDrive/ServiceRecommendation/Model/Model','wb'))

In [93]:
loadmodel = pickle.load(open('drive/MyDrive/ServiceRecommendation/Model/Model','rb'))

In [94]:
x =  int(loadmodel.predict([[27.6783400	,85.2883936]]))

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [95]:
print(encoder.inverse_transform(np.array(x).reshape(-1,1)))

[['1677414713802']]
